# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем таблицу и изучим общую информацию о данных

In [34]:
import pandas as pd
import seaborn as sns

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.**

In [35]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [37]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.**

In [38]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [39]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [40]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения.

**Выведем перечень уникальных значений столбца `children`.**

In [41]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [42]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [43]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [44]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [45]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [46]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [47]:
data['education'] = data['education'].str.lower()

In [48]:
data.duplicated().sum()

71

In [49]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [50]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [51]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [52]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [53]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [54]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Шаг 3. Исследуем данные

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [55]:
#Проведем категоризацию по количеству детей
def categorize_children(children):
    try:
        if children == 0:
            return 'Нет детей'
        elif 1 <= children <= 2:
            return 'Один или два ребенка'
        elif children>=3:
            return 'Более трех детей'
    except:
        pass
data['children_category'] = data['children'].apply(categorize_children)
data['children_category'].head(5)

0    Один или два ребенка
1    Один или два ребенка
2               Нет детей
3        Более трех детей
4               Нет детей
Name: children_category, dtype: object

In [56]:
# Сделаем сводную таблицу по новым категориям
children_debt = data.pivot_table(index = 'children_category', values = 'debt', aggfunc = ['count', 'mean'])
children_debt.columns = ['Количество клиентов в категории', 'Доля задолженности']
children_debt['Доля задолженности'] = children_debt['Доля задолженности'].apply('{:.2%}'.format)
children_debt

,Количество клиентов в категории,Доля задолженности
children_category,,
Более трех детей,380,8.16%
Нет детей,14091,7.54%
Один или два ребенка,6860,9.30%


**Вывод:** 
При категоризации клиентов на 3 группы по количеству детей, выявлено, что категория клиентов с тремя и более детьми не репрезентативна. По клиентам из данной категории невозможно сделать качественный вывод о возврате кредитов в срок. В двух оставшихся категориях наблюдается дифференциация. Так, клиенты, у которых нет детей имеют долю задолженностей ниже, в среднем на 1,8% чем у клиентов, которые имеют одного или два ребенка. 

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
# Также, сгруппируем данные по семейному положению и подсчитаем количество долгов в каждой группе
fam_debt = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'mean'])
fam_debt.columns = ['Количество клиентов в категории', 'Доля задолженности']
fam_debt['Доля задолженности'] = fam_debt['Доля задолженности'].apply('{:.2%}'.format)
fam_debt


,Количество клиентов в категории,Доля задолженности
family_status,,
Не женат / не замужем,2796,9.76%
в разводе,1189,7.06%
вдовец / вдова,951,6.62%
гражданский брак,4134,9.31%
женат / замужем,12261,7.56%


**Вывод:** 
В результате анализа, наименьшая задолженность по возврату кредита наблюдается в категории "вдова/вдовец", однако именно в этой категории меньше всего клиентов. Наибольшее количество задолженностей в категории клиентов, которые не женаты и не замужем, на 0,45% больше чем у клиентов живущих гражданским браком. Самой многочисленной является категория жентатых людей, там наблюдается средняя доля задолженностей, на уровне 7,56%. Клиенты, находящиеся в разводе, имеют вторую наименьшую долю задолженностей.  

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [58]:
# Расчитаем кол-во клиентов в каждой категории и сумму долгов
data.groupby('total_income_category').agg({'debt': ['count', 'sum'] })

debt      
                       count   sum
total_income_category             
A                         25     2
B                       5014   354
C                      15921  1353
D                        349    21
E                         22     2

In [59]:
# Найдем, сколько приходится долгов в каждой категории, в зависимости от количества клиентов в этой категории, в процентах
(data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count()) * 100 

total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
Name: debt, dtype: float64

**Вывод:** 
Статистика количества клиентов с очень высокой и очень низкой зарплатой - невелика. Поэтому нельзя утверждать, что эти клиенты с большей вероятностью вернут кредит. Процент задолженностей в категориях по величине дохода оказался практически равным во всех категориях. Однако, можно утверждать, что наибольшее количество клиентов, которые берут кредиты имеют средний доход от 50 до 200 тыс.руб.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [60]:
# Подсчитаем количество долгов в каждой группе по цели кредита и отсортируем данные по убыванию
purpose_debt = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'mean'])
purpose_debt.columns = ['Количество клиентов в категории', 'Доля задолженности']
purpose_debt['Доля задолженности'] = purpose_debt['Доля задолженности'].apply('{:.2%}'.format)
purpose_debt

,Количество клиентов в категории,Доля задолженности
purpose_category,,
операции с автомобилем,4279,9.35%
операции с недвижимостью,10751,7.26%
получение образования,3988,9.25%
проведение свадьбы,2313,7.91%


**Вывод:** 
Наименьшее количество задолженностей наблюдается в категории операций с недвижимостью, в то время как наибольшее количество задолженностей наблюдается в операциях с автомобилями и кредитах на образование, в которых практически одинаковый процент задолженностей. Проведение свадеб - категория со средними показателями задолженностей, в среднем, на 0,65% выше чем в категории с недвижимостью.

### Шаг 4: общий вывод.

В результате анализа данных платежеспособности клиентов банка, были выявлены следующие закономерности:
1. Категоризация клиентов по количеству детей, показала, что категория клиентов с тремя и более детьми не репрезентативна. По клиентам из данной категории невозможно сделать качественный вывод о возврате кредитов в срок. В двух оставшихся категориях наблюдается дифференциация. Так, клиенты, у которых нет детей имеют долю задолженностей ниже, в среднем на **1,8%** чем у клиентов, которые имеют одного или два ребенка. 

2. На основе анализа семейного положения клиентов выявлено, что наименьшая задолженность по возврату кредита наблюдается в категории "вдова/вдовец", однако именно в этой категории меньше всего клиентов. Наибольшее количество задолженностей в категории клиентов, которые не женаты и не замужем, на **0,45%** больше чем у клиентов живущих гражданским браком. Самой многочисленной является категория жентатых людей, там наблюдается средняя доля задолженностей, на уровне **7,56%**. Клиенты, находящиеся в разводе, имеют вторую наименьшую долю задолженностей.  

3. Статистика количества клиентов с очень высокой и очень низкой зарплатой - невелика. Поэтому нельзя утверждать, что эти клиенты с большей вероятностью вернут кредит. Процент задолженностей в категориях по величине дохода оказался практически равным во всех категориях. Однако, можно утверждать, что наибольшее количество клиентов, которые берут кредиты имеют средний доход от **50 до 200 тыс.руб**. 

4. При анализе на основе целей кредита: Наименьшее количество задолженностей наблюдается в категории операций с недвижимостью, в то время как наибольшее количество задолженностей наблюдается в операциях с автомобилями и кредитах на образование, в которых практически одинаковый процент задолженностей. Проведение свадеб - категория со средними показателями задолженностей, в среднем, на **0,65%** выше чем в категории с недвижимостью.

Таким образом, заказчику необходимо учитывать эти параметры при построении модели кредитного скоринга. 